In [175]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [11]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic//test.csv")
data_df = train_df.append(test_df) # The entire data: train + test.

In [17]:
# Cleaning name and extracting Title
data_df['Title'] = data_df['Name'].str.extract('([A-Za-z]+)\.', expand=True)
data_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,Mr
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,Dona
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,Mr


In [23]:
print(Counter(data_df['Title']))

Counter({'Mr': 757, 'Miss': 260, 'Mrs': 197, 'Master': 61, 'Rev': 8, 'Dr': 8, 'Col': 4, 'Ms': 2, 'Major': 2, 'Mlle': 2, 'Don': 1, 'Mme': 1, 'Lady': 1, 'Sir': 1, 'Capt': 1, 'Countess': 1, 'Jonkheer': 1, 'Dona': 1})


In [26]:
display(data_df.isnull().sum())

data_df[data_df['Fare'].isnull()]

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
Title             0
dtype: int64

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
152,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,Mr


In [70]:
missing_fare = data_df[(data_df['Embarked'] == 'S') & (data_df['Pclass'] == 3) & (data_df['Sex'] == 'male') & (data_df['Title'] == 'Mr') & (data_df['Age'] >= 50)].Fare.dropna().mean()
missing_fare = round(missing_fare, 4)
missing_fare

8.3019

In [76]:
data_df['Fare'].fillna(missing_fare, inplace=True)
display(data_df.isnull().sum())

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
Title             0
dtype: int64

In [77]:
data_df[data_df['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
61,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,Miss
829,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,Mrs


In [112]:
data_df[(data_df['Ticket'].notnull()) & (data_df['Ticket'].str.startswith('113')) & (data_df['Embarked'].notnull())]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
11,12,1.0,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,Miss
23,24,1.0,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5000,A6,S,Mr
35,36,0.0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,NaN,S,Mr
54,55,0.0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,1198,NaN,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0,1,2,113781,151.5500,C22 C26,S,Mr
323,1215,NaN,1,"Rowe, Mr. Alfred G",male,33.0,0,0,113790,26.5500,NaN,S,Mr
355,1247,NaN,1,"Julian, Mr. Henry Forbes",male,50.0,0,0,113044,26.0000,E60,S,Mr
403,1295,NaN,1,"Carrau, Mr. Jose Pedro",male,17.0,0,0,113059,47.1000,NaN,S,Mr


In [110]:
print(Counter(data_df[(data_df['Ticket'].notnull()) & (data_df['Ticket'].str.startswith('113')) & (data_df['Embarked'].notnull())]['Embarked']))

Counter({'S': 54, 'C': 10})


In [118]:
data_df[(data_df['Ticket'].notnull()) & (data_df['Ticket'].str.startswith('113')) & (data_df['Embarked'].notnull())].groupby(['Embarked'])['Fare'].mean()

Embarked
C    126.870840
S     57.162037
Name: Fare, dtype: float64

In [123]:
data_df[(data_df['Ticket'].notnull()) & (data_df['Ticket'].str.startswith('113')) & (data_df['Embarked'].notnull()) & (data_df['Embarked'] == 'C')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
54,55,0.0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C,Mr
377,378,0.0,1,"Widener, Mr. Harry Elkins",male,27.0,0,2,113503,211.5000,C82,C,Mr
452,453,0.0,1,"Foreman, Mr. Benjamin Laventall",male,30.0,0,0,113051,27.7500,C111,C,Mr
842,843,1.0,1,"Serepeca, Miss. Augusta",female,30.0,0,0,113798,31.0000,NaN,C,Miss
26,918,NaN,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,113509,61.9792,B36,C,Miss
74,966,NaN,1,"Geiger, Miss. Amalie",female,35.0,0,0,113503,211.5000,C130,C,Miss
75,967,NaN,1,"Keeping, Mr. Edwin",male,32.5,0,0,113503,211.5000,C132,C,Mr
131,1023,NaN,1,"Gracie, Col. Archibald IV",male,53.0,0,0,113780,28.5000,C51,C,Col
218,1110,NaN,1,"Widener, Mrs. George Dunton (Eleanor Elkins)",female,50.0,1,1,113503,211.5000,C80,C,Mrs
407,1299,NaN,1,"Widener, Mr. George Dunton",male,50.0,1,1,113503,211.5000,C80,C,Mr


In [124]:
data_df[(data_df['Ticket'].notnull()) & (data_df['Ticket'].str.startswith('113')) & (data_df['Embarked'].notnull()) & (data_df['Embarked'] == 'S')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S,Mrs
11,12,1.0,1,"Bonnell, Miss. Elizabeth",female,58.00,0,0,113783,26.55,C103,S,Miss
23,24,1.0,1,"Sloper, Mr. William Thompson",male,28.00,0,0,113788,35.50,A6,S,Mr
35,36,0.0,1,"Holverson, Mr. Alexander Oskar",male,42.00,1,0,113789,52.00,NaN,S,Mr
83,84,0.0,1,"Carrau, Mr. Francisco M",male,28.00,0,0,113059,47.10,NaN,S,Mr
137,138,0.0,1,"Futrelle, Mr. Jacques Heath",male,37.00,1,0,113803,53.10,C123,S,Mr
151,152,1.0,1,"Pears, Mrs. Thomas (Edith Wearne)",female,22.00,1,0,113776,66.60,C2,S,Mrs
166,167,1.0,1,"Chibnall, Mrs. (Edith Martha Bowerman)",female,NaN,0,1,113505,55.00,E33,S,Mrs
185,186,0.0,1,"Rood, Mr. Hugh Roscoe",male,NaN,0,0,113767,50.00,A32,S,Mr
252,253,0.0,1,"Stead, Mr. William Thomas",male,62.00,0,0,113514,26.55,C87,S,Mr


In [125]:
data_df['Embarked'].fillna('C', inplace=True) # TODO: If accuracy is low try to change 'S'
display(data_df.isnull().sum())

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          0
Title             0
dtype: int64

In [143]:
data_df[(data_df['Age'].isnull())]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
5,6,0.0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Mr
17,18,1.0,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,Mr
19,20,1.0,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C,Mrs
26,27,0.0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,Mr
28,29,1.0,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q,Miss
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,1300,NaN,3,"Riordan, Miss. Johanna Hannah""""",female,NaN,0,0,334915,7.7208,NaN,Q,Miss
410,1302,NaN,3,"Naughton, Miss. Hannah",female,NaN,0,0,365237,7.7500,NaN,Q,Miss
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,Mr
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,Mr


In [144]:
# Replacing rare titles with more common ones
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
          'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}
data_df.replace({'Title': mapping}, inplace=True)
titles = ['Dr', 'Master', 'Miss', 'Mr', 'Mrs', 'Rev']
for title in titles:
    age_to_impute = data_df[data_df['Age'].notnull()].groupby('Title')['Age'].median()[titles.index(title)]
    data_df.loc[(data_df['Age'].isnull()) & (data_df['Title'] == title), 'Age'] = age_to_impute
    

In [145]:
display(data_df.isnull().sum())

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          0
Title             0
dtype: int64

In [146]:
# Dropping Title feature
data_df.drop('Title', axis = 1, inplace = True)

In [147]:
data_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,30.0,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,30.0,0,0,359309,8.0500,NaN,S


In [148]:
data_df['Family_Size'] = data_df['Parch'] + data_df['SibSp']
data_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_Size
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,30.0,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,NaN,3,"Ware, Mr. Frederick",male,30.0,0,0,359309,8.0500,NaN,S,0


In [150]:
# Making Bins
data_df['FareBin'] = pd.qcut(data_df['Fare'], 5)

label = LabelEncoder()
data_df['FareBin_Code'] = label.fit_transform(data_df['FareBin'])
data_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_Size,FareBin,FareBin_Code
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,"(-0.001, 7.854]",0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,"(41.579, 512.329]",4
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,"(7.854, 10.5]",1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,"(41.579, 512.329]",4
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,"(7.854, 10.5]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,30.0,0,0,A.5. 3236,8.0500,NaN,S,0,"(7.854, 10.5]",1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0,"(41.579, 512.329]",4
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0,"(-0.001, 7.854]",0
416,1308,NaN,3,"Ware, Mr. Frederick",male,30.0,0,0,359309,8.0500,NaN,S,0,"(7.854, 10.5]",1


In [151]:
data_df.drop(['Fare'], 1, inplace=True)
data_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Cabin,Embarked,Family_Size,FareBin,FareBin_Code
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,NaN,S,1,"(-0.001, 7.854]",0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,C85,C,1,"(41.579, 512.329]",4
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,NaN,S,0,"(7.854, 10.5]",1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,C123,S,1,"(41.579, 512.329]",4
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,NaN,S,0,"(7.854, 10.5]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,30.0,0,0,A.5. 3236,NaN,S,0,"(7.854, 10.5]",1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,C105,C,0,"(41.579, 512.329]",4
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,NaN,S,0,"(-0.001, 7.854]",0
416,1308,NaN,3,"Ware, Mr. Frederick",male,30.0,0,0,359309,NaN,S,0,"(7.854, 10.5]",1


In [152]:
data_df['AgeBin'] = pd.qcut(data_df['Age'], 4)

label = LabelEncoder()
data_df['AgeBin_Code'] = label.fit_transform(data_df['AgeBin'])
data_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Cabin,Embarked,Family_Size,FareBin,FareBin_Code,AgeBin,AgeBin_Code
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,NaN,S,1,"(-0.001, 7.854]",0,"(0.169, 22.0]",0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,C85,C,1,"(41.579, 512.329]",4,"(36.0, 80.0]",3
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,C123,S,1,"(41.579, 512.329]",4,"(30.0, 36.0]",2
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,NaN,S,0,"(7.854, 10.5]",1,"(30.0, 36.0]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,30.0,0,0,A.5. 3236,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,C105,C,0,"(41.579, 512.329]",4,"(36.0, 80.0]",3
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,NaN,S,0,"(-0.001, 7.854]",0,"(36.0, 80.0]",3
416,1308,NaN,3,"Ware, Mr. Frederick",male,30.0,0,0,359309,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1


In [153]:
data_df.drop(['Age'], 1, inplace=True)
data_df

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Cabin,Embarked,Family_Size,FareBin,FareBin_Code,AgeBin,AgeBin_Code
0,1,0.0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,NaN,S,1,"(-0.001, 7.854]",0,"(0.169, 22.0]",0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,C85,C,1,"(41.579, 512.329]",4,"(36.0, 80.0]",3
2,3,1.0,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,C123,S,1,"(41.579, 512.329]",4,"(30.0, 36.0]",2
4,5,0.0,3,"Allen, Mr. William Henry",male,0,0,373450,NaN,S,0,"(7.854, 10.5]",1,"(30.0, 36.0]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,0,0,A.5. 3236,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,0,0,PC 17758,C105,C,0,"(41.579, 512.329]",4,"(36.0, 80.0]",3
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,0,0,SOTON/O.Q. 3101262,NaN,S,0,"(-0.001, 7.854]",0,"(36.0, 80.0]",3
416,1308,NaN,3,"Ware, Mr. Frederick",male,0,0,359309,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1


In [154]:
data_df['Sex'].replace(['male','female'],[0,1],inplace=True)
data_df

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Cabin,Embarked,Family_Size,FareBin,FareBin_Code,AgeBin,AgeBin_Code
0,1,0.0,3,"Braund, Mr. Owen Harris",0,1,0,A/5 21171,NaN,S,1,"(-0.001, 7.854]",0,"(0.169, 22.0]",0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,0,PC 17599,C85,C,1,"(41.579, 512.329]",4,"(36.0, 80.0]",3
2,3,1.0,3,"Heikkinen, Miss. Laina",1,0,0,STON/O2. 3101282,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,0,113803,C123,S,1,"(41.579, 512.329]",4,"(30.0, 36.0]",2
4,5,0.0,3,"Allen, Mr. William Henry",0,0,0,373450,NaN,S,0,"(7.854, 10.5]",1,"(30.0, 36.0]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",0,0,0,A.5. 3236,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",1,0,0,PC 17758,C105,C,0,"(41.579, 512.329]",4,"(36.0, 80.0]",3
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",0,0,0,SOTON/O.Q. 3101262,NaN,S,0,"(-0.001, 7.854]",0,"(36.0, 80.0]",3
416,1308,NaN,3,"Ware, Mr. Frederick",0,0,0,359309,NaN,S,0,"(7.854, 10.5]",1,"(22.0, 30.0]",1


In [155]:
data_df.drop(['Name', 'SibSp', 'Parch', 'Ticket', 'Cabin',
               'FareBin', 'AgeBin'], axis = 1, inplace = True)
data_df

,PassengerId,Survived,Pclass,Sex,Embarked,Family_Size,FareBin_Code,AgeBin_Code
0,1,0.0,3,0,S,1,0,0
1,2,1.0,1,1,C,1,4,3
2,3,1.0,3,1,S,0,1,1
3,4,1.0,1,1,S,1,4,2
4,5,0.0,3,0,S,0,1,2
...,...,...,...,...,...,...,...,...
413,1305,NaN,3,0,S,0,1,1
414,1306,NaN,1,1,C,0,4,3
415,1307,NaN,3,0,S,0,0,3
416,1308,NaN,3,0,S,0,1,1


In [158]:
data_df['Embarked'].replace(['S','C','Q'],[1,2,3],inplace=True)
data_df

,PassengerId,Survived,Pclass,Sex,Embarked,Family_Size,FareBin_Code,AgeBin_Code
0,1,0.0,3,0,1,1,0,0
1,2,1.0,1,1,2,1,4,3
2,3,1.0,3,1,1,0,1,1
3,4,1.0,1,1,1,1,4,2
4,5,0.0,3,0,1,0,1,2
...,...,...,...,...,...,...,...,...
413,1305,NaN,3,0,1,0,1,1
414,1306,NaN,1,1,2,0,4,3
415,1307,NaN,3,0,1,0,0,3
416,1308,NaN,3,0,1,0,1,1


In [159]:
train_df = data_df[:891]
test_df = data_df[891:]

In [161]:
train_df

,PassengerId,Survived,Pclass,Sex,Embarked,Family_Size,FareBin_Code,AgeBin_Code
0,1,0.0,3,0,1,1,0,0
1,2,1.0,1,1,2,1,4,3
2,3,1.0,3,1,1,0,1,1
3,4,1.0,1,1,1,1,4,2
4,5,0.0,3,0,1,0,1,2
...,...,...,...,...,...,...,...,...
886,887,0.0,2,0,1,0,2,1
887,888,1.0,1,1,1,0,3,0
888,889,0.0,3,1,1,3,3,0
889,890,1.0,1,0,2,0,3,1


In [166]:
test_df.drop(['Survived'], 1, inplace=True)
test_df

,PassengerId,Pclass,Sex,Embarked,Family_Size,FareBin_Code,AgeBin_Code
0,892,3,0,3,0,0,2
1,893,3,1,1,1,0,3
2,894,2,0,3,0,1,3
3,895,3,0,1,0,1,1
4,896,3,1,1,2,2,0
...,...,...,...,...,...,...,...
413,1305,3,0,1,0,1,1
414,1306,1,1,2,0,4,3
415,1307,3,0,1,0,0,3
416,1308,3,0,1,0,1,1


In [167]:
X = train_df.drop('Survived', 1)
y = train_df['Survived']
X_test = test_df.copy()

In [169]:
std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)
X_test = std_scaler.transform(X_test)

In [172]:
n_neighbors = [6,7,8,9,10,11,12,14,16,18,20,22]
algorithm = ['auto']
weights = ['uniform', 'distance']
leaf_size = list(range(1,50,5))
hyperparams = {'algorithm': algorithm, 'weights': weights, 'leaf_size': leaf_size, 
               'n_neighbors': n_neighbors}
gd=GridSearchCV(estimator = KNeighborsClassifier(), param_grid = hyperparams, verbose=True, 
                cv=10, scoring = "roc_auc")
gd.fit(X, y)
print(gd.best_score_)
print(gd.best_estimator_)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits
0.8589543049543049
KNeighborsClassifier(leaf_size=1, n_neighbors=18)


In [178]:
gd.best_estimator_.fit(X, y)
y_pred = gd.best_estimator_.predict(X_test)

In [184]:
y_pred

array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [189]:
temp = pd.DataFrame(pd.read_csv("/kaggle/input/titanic//test.csv")['PassengerId'])
temp['Survived'] = y_pred
temp['Survived'] = temp['Survived'].astype(int)
temp.to_csv("submission_knn_1.csv", index = False)
temp

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [173]:
knn = KNeighborsClassifier(algorithm='auto', leaf_size=26, metric='minkowski', 
                           metric_params=None, n_jobs=1, n_neighbors=6, p=2, 
                           weights='uniform')
knn.fit(X, y)

KNeighborsClassifier(leaf_size=26, n_jobs=1, n_neighbors=6)

In [177]:
y_pred = knn.predict(X)
acc = accuracy_score(y, y_pred)
acc

0.8507295173961841